In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("SLU Opportunity Wise Data-1710158595043.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(r"\s+", "_", regex = True)

In [3]:
print(df.shape)

(8558, 16)


In [4]:
print(df.isna().sum())

learner_signup_datetime       0
opportunity_id                0
opportunity_name              0
opportunity_category          0
opportunity_end_date          0
first_name                    0
date_of_birth                 0
gender                        0
country                       0
institution_name              5
current/intended_major        5
entry_created_at              0
status_description            0
status_code                   0
apply_date                    0
opportunity_start_date     3794
dtype: int64


In [5]:
df['start_date_missing'] = df['opportunity_start_date'].isna().map({True: 'Yes', False: 'No'})

In [6]:
df.head()

,learner_signup_datetime,opportunity_id,opportunity_name,opportunity_category,opportunity_end_date,first_name,date_of_birth,gender,country,institution_name,current/intended_major,entry_created_at,status_description,status_code,apply_date,opportunity_start_date,start_date_missing
0,06/14/2023 12:30:35,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,06/29/2024 18:52:39,Faria,01/12/2001,Female,Pakistan,Nwihs,Radiology,03/11/2024 12:01:41,Started,1080,06/14/2023 12:36:09,11/03/2022 18:30:39,No
1,05/01/2023 05:29:16,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,06/29/2024 18:52:39,Poojitha,08/16/2000,Female,India,SAINT LOUIS,Information Systems,03/11/2024 12:01:41,Started,1080,05/01/2023 06:08:21,11/03/2022 18:30:39,No
2,04/09/2023 20:35:08,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,06/29/2024 18:52:39,Emmanuel,01/27/2002,Male,United States,Illinois Institute of Technology,Computer Science,03/11/2024 12:01:41,Started,1080,05/11/2023 1085640:21:29,11/03/2022 18:30:39,No
3,08/29/2023 05:20:03,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,06/29/2024 18:52:39,Amrutha Varshini,11/01/1999,Female,United States,Saint Louis University,Information Systems,03/11/2024 12:01:41,Team Allocated,1070,10/09/2023 22:02:42,11/03/2022 18:30:39,No
4,01/06/2023 15:26:36,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,06/29/2024 18:52:39,Vinay Varshith,04/19/2000,Male,United States,Saint Louis University,Computer Science,03/11/2024 12:01:41,Started,1080,01/06/2023 15:40:10,11/03/2022 18:30:39,No


In [7]:
date_cols = ['learner_signup_datetime', 'apply_date', 'opportunity_start_date', 'opportunity_end_date', 'entry_created_at', 'date_of_birth'] 
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors = 'coerce' )

In [8]:
mask_apply = df['apply_date'] < df['learner_signup_datetime']
df.loc[mask_apply, 'apply_date'] = df.loc[mask_apply, 'learner_signup_datetime']

In [9]:
df['opportunity_end_date'] = df.apply(
    lambda x: x['opportunity_end_date'] if pd.notna(x['opportunity_start_date']) 
    and pd.notna(x['opportunity_end_date'])
    and x['opportunity_end_date']>= x['opportunity_start_date'] else np.nan, axis=1
)

In [10]:
today = pd.Timestamp.today()
df['age'] = (today - df['date_of_birth']).dt.days // 365

In [11]:
bins = [0, 17, 24, 34, 44, 54, 64, 120]
labels = ['<18','18-24','25-34','35-44','45-54','55-64','65+']
df['age_band'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)

In [12]:
df['apply_lag'] = (df['apply_date'] - df['learner_signup_datetime']).dt.days

In [13]:
df['start_lag'] = (df['opportunity_start_date'] - df['apply_date']).dt.days
df.loc[df['start_lag'] < 0, 'start_lag'] = pd.NA

In [14]:
df['opportunity_duration'] = (df['opportunity_end_date'] - df['opportunity_start_date']).dt.days
df.loc[df['opportunity_duration'] < 0, 'opportunity_duration'] = pd.NA

In [15]:
df['signup_cohort'] = df['learner_signup_datetime'].dt.to_period('M').astype(str)

In [16]:
df.head()

,learner_signup_datetime,opportunity_id,opportunity_name,opportunity_category,opportunity_end_date,first_name,date_of_birth,gender,country,institution_name,...,status_code,apply_date,opportunity_start_date,start_date_missing,age,age_band,apply_lag,start_lag,opportunity_duration,signup_cohort
0,2023-06-14 12:30:35,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Faria,2001-01-12,Female,Pakistan,Nwihs,...,1080,2023-06-14 12:36:09,2022-11-03 18:30:39,No,24,18-24,0.0,NaN,604.0,2023-06
1,2023-05-01 05:29:16,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Poojitha,2000-08-16,Female,India,SAINT LOUIS,...,1080,2023-05-01 06:08:21,2022-11-03 18:30:39,No,25,25-34,0.0,NaN,604.0,2023-05
2,2023-04-09 20:35:08,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Emmanuel,2002-01-27,Male,United States,Illinois Institute of Technology,...,1080,NaT,2022-11-03 18:30:39,No,23,18-24,NaN,NaN,604.0,2023-04
3,2023-08-29 05:20:03,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Amrutha Varshini,1999-11-01,Female,United States,Saint Louis University,...,1070,2023-10-09 22:02:42,2022-11-03 18:30:39,No,25,25-34,41.0,NaN,604.0,2023-08
4,2023-01-06 15:26:36,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Vinay Varshith,2000-04-19,Male,United States,Saint Louis University,...,1080,2023-01-06 15:40:10,2022-11-03 18:30:39,No,25,25-34,0.0,NaN,604.0,2023-01


In [17]:
df = df.drop(columns=['first_name','date_of_birth','current/intended_major'], errors='ignore')

In [18]:
completed_codes = [1080, 1070, 1120] #started,team allocated,rewards
dropped_codes = [1030, 1050, 1110, 1040] #rejected, dropped, waitlisted
df['completion_flag'] = df['status_code'].apply(lambda x:1 if x in completed_codes else 0)

In [19]:
df['dropoff_flag'] = df['status_code'].apply(lambda x:'Yes' if x in dropped_codes else 'No')

In [20]:
stage_map = {1080: 1, 1070:2, 1040:3, 1050:4, 1110:5, 1120:6, 1030:0}
df['stage_reached'] = df['status_code'].map(stage_map)

In [21]:
outcome_map ={
    1080:'Started',
    1070:'Team Allocated',
    1040:'Waitlisted',
    1110:'Withdrawn',
    1120:'Awarded',
    1030:'Rejected',
    1050:'Dropped Out'
}
df['final_outcome'] = df['status_code'].map(outcome_map)

In [22]:
df.head()

,learner_signup_datetime,opportunity_id,opportunity_name,opportunity_category,opportunity_end_date,gender,country,institution_name,entry_created_at,status_description,...,age,age_band,apply_lag,start_lag,opportunity_duration,signup_cohort,completion_flag,dropoff_flag,stage_reached,final_outcome
0,2023-06-14 12:30:35,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Female,Pakistan,Nwihs,2024-03-11 12:01:41,Started,...,24,18-24,0.0,NaN,604.0,2023-06,1,No,1.0,Started
1,2023-05-01 05:29:16,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Female,India,SAINT LOUIS,2024-03-11 12:01:41,Started,...,25,25-34,0.0,NaN,604.0,2023-05,1,No,1.0,Started
2,2023-04-09 20:35:08,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Male,United States,Illinois Institute of Technology,2024-03-11 12:01:41,Started,...,23,18-24,NaN,NaN,604.0,2023-04,1,No,1.0,Started
3,2023-08-29 05:20:03,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Female,United States,Saint Louis University,2024-03-11 12:01:41,Team Allocated,...,25,25-34,41.0,NaN,604.0,2023-08,1,No,2.0,Team Allocated
4,2023-01-06 15:26:36,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Male,United States,Saint Louis University,2024-03-11 12:01:41,Started,...,25,25-34,0.0,NaN,604.0,2023-01,1,No,1.0,Started


In [23]:
df.dropna()

,learner_signup_datetime,opportunity_id,opportunity_name,opportunity_category,opportunity_end_date,gender,country,institution_name,entry_created_at,status_description,...,age,age_band,apply_lag,start_lag,opportunity_duration,signup_cohort,completion_flag,dropoff_flag,stage_reached,final_outcome
1442,2023-08-19 15:27:36,00000000-0GNT-FT74-MZT8-93VC0G,Digital Marketing,Internship,2024-03-11 18:00:00,Female,China,珠海一附国际部,2024-03-11 12:01:57,Team Allocated,...,15,<18,7.0,133.0,63.0,2023-08,1,No,2.0,Team Allocated
1454,2023-08-15 12:41:36,00000000-0GNT-FT74-MZT8-93VC0G,Digital Marketing,Internship,2024-03-11 18:00:00,Female,United Kingdom,Caleb university,2024-03-11 12:01:57,Team Allocated,...,27,25-34,0.0,145.0,63.0,2023-08,1,No,2.0,Team Allocated
1460,2023-12-18 15:38:37,00000000-0GNT-FT74-MZT8-93VC0G,Digital Marketing,Internship,2024-03-11 18:00:00,Male,India,DEEN DAYAL UPADHYAYA GORAKHPUR UNIVERSITY,2024-03-11 12:01:57,Dropped Out,...,25,25-34,0.0,20.0,63.0,2023-12,0,Yes,4.0,Dropped Out
1463,2023-10-31 09:30:41,00000000-0GNT-FT74-MZT8-93VC0G,Digital Marketing,Internship,2024-03-11 18:00:00,Male,Nigeria,Ahmadu Bello university Zaria,2024-03-11 12:01:57,Dropped Out,...,27,25-34,57.0,11.0,63.0,2023-10,0,Yes,4.0,Dropped Out
1466,2023-08-22 03:09:15,00000000-0GNT-FT74-MZT8-93VC0G,Digital Marketing,Internship,2024-03-11 18:00:00,Male,United States,Saint Louis University,2024-03-11 12:01:57,Dropped Out,...,32,25-34,123.0,15.0,63.0,2023-08,0,Yes,4.0,Dropped Out
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8288,2023-12-30 02:46:32,00000000-100J-PM3A-0WJ8-5T68M5,Jump Start: Developing your Emotional Intellig...,Course,2025-12-24 03:34:00,Male,Nigeria,IHERIS UNIVERSITY TOGO,2024-03-11 12:03:11,Team Allocated,...,27,25-34,28.0,1.0,695.0,2023-12,1,No,2.0,Team Allocated
8291,2023-05-03 10:49:06,00000000-100J-PM3A-0WJ8-5T68M5,Jump Start: Developing your Emotional Intellig...,Course,2025-12-24 03:34:00,Female,India,Saint louis University,2024-03-11 12:03:11,Team Allocated,...,24,18-24,267.0,2.0,695.0,2023-05,1,No,2.0,Team Allocated
8301,2023-12-29 21:42:40,00000000-100J-PM3A-0WJ8-5T68M5,Jump Start: Developing your Emotional Intellig...,Course,2025-12-24 03:34:00,Male,United States,Saint Louis University,2024-03-11 12:03:12,Team Allocated,...,23,18-24,30.0,0.0,695.0,2023-12,1,No,2.0,Team Allocated
8304,2023-06-16 09:40:36,00000000-100J-PM3A-0WJ8-5T68M5,Jump Start: Developing your Emotional Intellig...,Course,2025-12-24 03:34:00,Male,India,Saint Louis University,2024-03-11 12:03:12,Team Allocated,...,23,18-24,223.0,2.0,695.0,2023-06,1,No,2.0,Team Allocated


In [24]:
print('Duplicate rows:', df.duplicated().sum())

Duplicate rows: 0


In [25]:
print(df.dtypes)

learner_signup_datetime    datetime64[ns]
opportunity_id                     object
opportunity_name                   object
opportunity_category               object
opportunity_end_date       datetime64[ns]
gender                             object
country                            object
institution_name                   object
entry_created_at           datetime64[ns]
status_description                 object
status_code                         int64
apply_date                 datetime64[ns]
opportunity_start_date     datetime64[ns]
start_date_missing                 object
age                                 int64
age_band                         category
apply_lag                         float64
start_lag                         float64
opportunity_duration              float64
signup_cohort                      object
completion_flag                     int64
dropoff_flag                       object
stage_reached                     float64
final_outcome                     

In [26]:
df['institution_name'] = df['institution_name'].str.title()
df['country'] = df['country'].str.title()

In [27]:
df.shape

(8558, 24)

In [28]:
df.columns

Index(['learner_signup_datetime', 'opportunity_id', 'opportunity_name',
       'opportunity_category', 'opportunity_end_date', 'gender', 'country',
       'institution_name', 'entry_created_at', 'status_description',
       'status_code', 'apply_date', 'opportunity_start_date',
       'start_date_missing', 'age', 'age_band', 'apply_lag', 'start_lag',
       'opportunity_duration', 'signup_cohort', 'completion_flag',
       'dropoff_flag', 'stage_reached', 'final_outcome'],
      dtype='object')

In [29]:
df.isna().sum()

learner_signup_datetime     295
opportunity_id                0
opportunity_name              0
opportunity_category          0
opportunity_end_date       5612
gender                        0
country                       0
institution_name              5
entry_created_at              0
status_description            0
status_code                   0
apply_date                  307
opportunity_start_date     4637
start_date_missing            0
age                           0
age_band                      0
apply_lag                   534
start_lag                  6816
opportunity_duration       5612
signup_cohort                 0
completion_flag               0
dropoff_flag                  0
stage_reached               105
final_outcome               105
dtype: int64

In [30]:
df.describe()

,learner_signup_datetime,opportunity_end_date,entry_created_at,status_code,apply_date,opportunity_start_date,age,apply_lag,start_lag,opportunity_duration,completion_flag,stage_reached
count,8263,2946,8558,8558.000000,8251,3921,8558.000000,8024.000000,1742.000000,2946.000000,8558.000000,8453.000000
mean,2023-09-28 06:04:46.473677568,2024-08-01 14:54:00.770196736,2024-03-11 12:02:27.589273088,1052.225987,2023-11-24 11:09:55.480911616,2023-07-24 01:05:46.093343744,25.501753,56.656032,72.616533,439.687373,0.475812,1.267952
min,2023-01-05 16:32:31,2024-03-11 18:00:00,2024-03-11 12:01:41,1010.000000,2023-01-05 16:32:31,2022-11-03 18:30:39,6.000000,0.000000,0.000000,38.000000,0.000000,0.000000
25%,2023-07-09 06:10:29,2024-04-19 18:29:00,2024-03-11 12:02:04,1030.000000,2023-09-11 14:08:46.500000,2022-11-03 18:30:39,23.000000,0.000000,18.000000,63.000000,0.000000,0.000000
50%,2023-09-21 01:46:40,2024-06-29 18:52:39,2024-03-11 12:02:27,1050.000000,2024-01-06 05:21:09,2024-01-08 03:30:46,25.000000,4.000000,40.000000,604.000000,0.000000,1.000000
75%,2024-01-10 05:26:48.500000,2024-06-29 18:52:39,2024-03-11 12:02:52,1070.000000,2024-02-03 02:13:17.500000,2024-01-28 18:30:00,27.000000,92.000000,132.750000,604.000000,1.000000,2.000000
max,2024-03-11 10:43:44,2025-12-24 03:34:00,2024-03-11 12:03:14,1120.000000,2024-03-11 11:07:38,2024-05-31 07:04:54,59.000000,426.000000,322.000000,913.000000,1.000000,6.000000
std,NaN,NaN,NaN,21.665207,NaN,NaN,4.376911,89.071643,67.187675,265.205978,0.499444,1.302010


In [31]:
df.mode().iloc[0]

learner_signup_datetime                                  2023-01-05 16:33:15
opportunity_id                                00000000-0GN2-A0AY-7XK8-C5FZPP
opportunity_name           Career Essentials: Getting Started with Your P...
opportunity_category                                              Internship
opportunity_end_date                                     2024-06-29 18:52:39
gender                                                                  Male
country                                                        United States
institution_name                                      Saint Louis University
entry_created_at                                         2024-03-11 12:02:04
status_description                                                  Rejected
status_code                                                           1030.0
apply_date                                               2023-01-05 16:33:16
opportunity_start_date                                   2022-11-03 18:30:39

In [32]:
df['completion_flag'].value_counts()

completion_flag
0    4486
1    4072
Name: count, dtype: int64

In [33]:
df['completion_flag'].value_counts(normalize=True) *100

completion_flag
0    52.418789
1    47.581211
Name: proportion, dtype: float64

In [34]:
df['stage_reached'].value_counts()

stage_reached
0.0    3569
2.0    3276
1.0     767
4.0     617
3.0     109
5.0      86
6.0      29
Name: count, dtype: int64

In [35]:
df['stage_reached'].value_counts(normalize=True)*100

stage_reached
0.0    42.221696
2.0    38.755471
1.0     9.073702
4.0     7.299184
3.0     1.289483
5.0     1.017390
6.0     0.343073
Name: proportion, dtype: float64

In [36]:
df.groupby('gender')['completion_flag'].value_counts()

gender                 completion_flag
Don't want to specify  1                    11
                       0                     4
Female                 0                  1798
                       1                  1724
Male                   0                  2682
                       1                  2336
Other                  0                     2
                       1                     1
Name: count, dtype: int64

In [37]:
df.groupby('gender')['completion_flag'].value_counts(normalize=True)*100

gender                 completion_flag
Don't want to specify  1                  73.333333
                       0                  26.666667
Female                 0                  51.050539
                       1                  48.949461
Male                   0                  53.447589
                       1                  46.552411
Other                  0                  66.666667
                       1                  33.333333
Name: proportion, dtype: float64

In [38]:
df.groupby('age_band')['completion_flag'].value_counts()

C:\Users\kanko\AppData\Local\Temp\ipykernel_6304\3884447499.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('age_band')['completion_flag'].value_counts()


age_band  completion_flag
<18       1                    63
          0                    27
18-24     0                  1936
          1                  1862
25-34     0                  2348
          1                  1951
35-44     1                   158
          0                   141
45-54     0                    33
          1                    33
55-64     1                     5
          0                     1
65+       0                     0
          1                     0
Name: count, dtype: int64

In [39]:
df.groupby('age_band')['completion_flag'].value_counts(normalize=True)*100

C:\Users\kanko\AppData\Local\Temp\ipykernel_6304\175304136.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('age_band')['completion_flag'].value_counts(normalize=True)*100


age_band  completion_flag
<18       1                  70.000000
          0                  30.000000
18-24     0                  50.974197
          1                  49.025803
25-34     0                  54.617353
          1                  45.382647
35-44     1                  52.842809
          0                  47.157191
45-54     0                  50.000000
          1                  50.000000
55-64     1                  83.333333
          0                  16.666667
65+       0                   0.000000
          1                   0.000000
Name: proportion, dtype: float64

In [40]:
df.groupby('institution_name')['completion_flag'].value_counts()

institution_name                           completion_flag
Aa                                         0                  1
Aacharya Ng Ranga Agricultural University  1                  1
Aamusted                                   1                  1
Abc                                        0                  1
Abc Inter College                          0                  1
                                                             ..
ثانوية ابن سينا التأهيلية                  1                  1
广州市实验外语学校                                  1                  1
珠海一附国际部                                    1                  2
珠海市一附属实验学校                                 1                  1
长沙学院                                       1                  1
Name: count, Length: 2203, dtype: int64

In [41]:
df.groupby('institution_name')['completion_flag'].value_counts(normalize=True)*100

institution_name                           completion_flag
Aa                                         0                  100.0
Aacharya Ng Ranga Agricultural University  1                  100.0
Aamusted                                   1                  100.0
Abc                                        0                  100.0
Abc Inter College                          0                   50.0
                                                              ...  
ثانوية ابن سينا التأهيلية                  1                  100.0
广州市实验外语学校                                  1                  100.0
珠海一附国际部                                    1                  100.0
珠海市一附属实验学校                                 1                  100.0
长沙学院                                       1                  100.0
Name: proportion, Length: 2203, dtype: float64

In [42]:
df.corr(numeric_only=True)['completion_flag'].sort_values(ascending=False)

completion_flag         1.000000
status_code             0.880165
opportunity_duration    0.464269
stage_reached           0.423699
start_lag               0.116511
apply_lag              -0.017519
age                    -0.017935
Name: completion_flag, dtype: float64

In [43]:
df.corr(numeric_only=True)

,status_code,age,apply_lag,start_lag,opportunity_duration,completion_flag,stage_reached
status_code,1.000000,-0.017098,-0.054239,0.104166,0.443293,0.880165,0.680207
age,-0.017098,1.000000,-0.016937,0.135634,0.001216,-0.017935,0.005942
apply_lag,-0.054239,-0.016937,1.000000,-0.264199,-0.137048,-0.017519,0.002801
start_lag,0.104166,0.135634,-0.264199,1.000000,-0.111794,0.116511,-0.088360
opportunity_duration,0.443293,0.001216,-0.137048,-0.111794,1.000000,0.464269,-0.462941
completion_flag,0.880165,-0.017935,-0.017519,0.116511,0.464269,1.000000,0.423699
stage_reached,0.680207,0.005942,0.002801,-0.088360,-0.462941,0.423699,1.000000


In [25]:
df.to_excel('cleaned_dataset.xlsx', index=False)